#  Muchas reservas son canceladas a última hora ¿cómo implementar un cambio en la política de cancelación?
> Analizaremos el caso de un hotel que el +40% de sus reservas son canceladas el mismo día de la llegada y cuya única política de cancelación ofrece pago en destino sin penalidad.

El **Hotel ABC** tiene una política de cancelación 100% flexible.  En la cual los huéspedes pueden realizar cancelaciones en cualquier momento sin tener una penalidad y realizar el pago al momento de arribar al hotel.

<table cellspacing="5" cellpadding="5">
    <tr>
        <td><img src="files/output_53_0.png" width="350"/></td>
        <td><img src="files/output_60_0.png" width="350"/></td> 
    </tr>
</table>

Esta política genera un comportamiento de alto porcentaje de cancelación (cerca del 40%, 4 de cada 10 reservas son canceladas <span style='font-size:15px;'>&#128563;</span>), cerca del 60% son canceladas con 3 días de antelación y el 43% el mismo día de la llegada.  Son reservas efímeras, más del 50% tienen un tiempo de vida (mientras se crean y cancelan) de dos (2) días.  Esta entropía genera una gran incertidumbre sobre las expectativas de ingresos y dificultan la gestión de *revenue*, llevándola al "micro-management" de precios y promociones de última hora a fin de apagar el incendio. <span style='font-size:15px;'>&#128658;</span>

<table cellspacing="5" cellpadding="5">
    <tr>
        <td><img src="files/output_5_0.png" width="700"/></td>
    </tr>
</table>

Por suerte, la estrategia en curso ha venido desempeñándose con éxito, con una tasa promedio de ocupación del primer semestre cerca al 90%.  El objetivo aquí se centra en la optimización de la estrategia, buscando mantener una seguridad en los ingresos con las reservas confirmadas a escasos días de la llegada.

### **¿Cómo optimizar la estrategia?**
Lo primero es averiguar ¿qué tanto los huéspedes realizan sus pagos con tarjeta de crédito?.  A pesar que no se exija un pago por adelantado con cobro directo a la tarjeta de crédito; aún sería posible que los huéspedes prefieran usarla al momento de llegar al hotel.  En este caso, migrar hacia una política que exija el cobro anticipado, será un paso que no generaría un potencial rechazo en la conversión de reservas dentro de su nicho de mercado.  Para responder a esta hipótesis, analizaremos los medios de pagos que han sido utilizados en los últimos 5 meses.

Se han creado dos archivos de datos obtenidos por medio consultas *en-línea* al software hotelero:

* <code>export_payment_list.csv</code> Es una tabla de pagos realizados, contiene el **medio de pago** usado y el **monto**.
* <code>rsv_df.csv</code> Es una tabla de todas las reservas, contiene detalles de cada reserva como la **agencia** que vendió la reserva y el **país** del huésped.

Al unir los dos conjuntos de datos, podremos determinan no solo cual es el medio de pago más utilizado; sino también, como se dan estos pagos por canal de venta, país o perfil de la reserva, formulando la respuesta a la hipótesis planteada.

#### **¿Qué tanto los huéspedes usan como medio de pago la tarjeta de crédito?**
Solo el 20% lo hace, es decir, **1 de cada 5 reservas son pagadas por medio de la tarjeta de crédito**.  Es decir, que si se desea implementar el cobro por anticipado de las reservas, un 80% de los huéspedes estaría fuera.  Allí hay un riesgo de fallo en la implementación inmediata, que podría llevar a un decrecimiento en la tasa de ocupación e ingresos.

In [6]:
import pandas as pd

# Read data sources.  Payments & Reservations
payments = pd.read_csv('export_payment_list.csv', sep=';')
df_rsvs = pd.read_csv('data/rsv_df.csv', parse_dates=[5, 7, 15, 16],  dayfirst=True)
df_rsvs.fillna(value={'agency':'Hotel ABC', 'customer_country':'Unknown'}, inplace=True)

# Inner Join payments & reservations dataframes
payments_rsv = payments.merge(df_rsvs, left_on='Reserva', right_on='id_human', how='inner', indicator=True)
payments_rsv.fillna({'agency':'Unknown', 'Tipo':'Unknown', 'customer_country':'Unknown'}, inplace=True)

payments_rsv_type = payments_rsv.groupby('Tipo', as_index=False)[['Cantidad']].sum().\
                    set_index('Tipo').sort_values(by='Cantidad', ascending=False)
payments_rsv_type.loc['Total'] = payments_rsv_type.sum()
payments_rsv_type['%'] = (payments_rsv_type['Cantidad'] / payments_rsv_type.iloc[-1,0])*100
payments_rsv_type.style.format({"Cantidad": "${:20,.0f}", "%": "{:20,.2f}%"})

,Cantidad,%
Tipo,,
Efectivo,"$ 70,008,513",32.31%
Tarjeta De Débito,"$ 64,146,620",29.60%
Tarjeta De Crédito,"$ 44,566,702",20.57%
Transferencia Bancaria,"$ 35,507,330",16.39%
Pago Online,"$ 2,452,420",1.13%
Total,"$ 216,681,585",100.00%


#### **¿Cómo se usa la tarjeta de crédito por canales?**
Al desagrupar ese 20% por canales de venta, tenemos que el 12.5% de los pagos por medio de tarjeta de crédito lo realizan huéspedes provenientes de Booking.com y un 7.8% por la venta directa del hotel, cuarto y séptimo lugar en la cantidad total de pagos.  Mientras que los tres primeros tipos de pagos son de contado (Tarjeta Débito y Efectivo), corresponden a cerca del 50% de la cantidad total.  Para el caso de Booking.com (que es el canal por donde se iniciaría la aplicación de una política más restrictiva), alrededor de uno (1) de cada cuatro (4) pesos que se pagan es por medio de tarjeta de crédito.

In [22]:
# Groupby type of payments by agency
payments_rsv_type_agency = payments_rsv.groupby(['agency', 'Tipo'])[['Cantidad']].sum().\
                                                sort_values(by='Cantidad', ascending=False)
payments_rsv_type_agency['%'] = (payments_rsv_type_agency['Cantidad'] / payments_rsv_type_agency\
                                 ['Cantidad'].sum())*100
payments_rsv_type_agency.style.format({"Cantidad": "${:20,.0f}", "%": "{:20,.2f}%"})\
                                        .bar(subset=["Cantidad",], color='lightgreen')

#### **¿Qué tipo de clientes pagan con tarjeta de crédito?**
El 25% de las reservas que provienen de Booking.com son pagadas con tarjeta de crédito por una pareja colombiana, que crean su reserva con una semana de anticipación, tienen una estadía por dos noches a un precio promedio de $170.000.  4 de cada 10 pagos con tarjeta de crédito lo realiza un huésped colombiano, es decir que, **la mayoría de los clientes que pagan con tarjeta de crédito son extranjeros**.

In [42]:
customer_pay_by_credit_card = payments_rsv[payments_rsv['Tipo'] == 'Tarjeta De Crédito']
customer_pay_by_credit_card_agency = \
customer_pay_by_credit_card.groupby(['agency', 'customer_country'])\
                                    [['Cantidad', 'los', 'bWindow', 'adr', 'occupancy']].\
                                    agg({'Cantidad':'sum', 'los':'mean', 'bWindow':'mean',\
                                         'adr':'mean', 'occupancy':'mean'})\
                                        .sort_values(by='Cantidad', ascending=False)
customer_pay_by_credit_card_agency['%'] = (customer_pay_by_credit_card_agency['Cantidad']\
                                           / customer_pay_by_credit_card_agency['Cantidad'].sum())*100
customer_pay_by_credit_card_agency = customer_pay_by_credit_card_agency.\
                                     reindex(columns=['Cantidad', '%', 'los', 'bWindow', 'adr', 'occupancy']) 
customer_pay_by_credit_card_agency.style.format({"Cantidad":"${:20,.0f}", "adr":"${:20,.0f}",\
                        "%":"{:20,.2f}%", "los":"{:20,.1f}", "bWindow":"{:20,.1f}", "occupancy":"{:20,.1f}"})\
                                                .bar(subset=["Cantidad",], color='lightgreen')

#### **¿Qué tipo de clientes pagan con otros medios?**
El 77% de los pagos hechos por medios de pagos diferentes a tarjeta de crédito proviene de una pareja colombiana que reserva entre 10 y 6 días antes de su llegada, por 2 noches y a un precio promedio de $185.000 pesos por noche.  3 de cada 4 pesos pagados por medios diferente a tarjeta de crédito son de huéspedes colombianos.  Lo que quiere decir que **la minoría de los clientes que pagan por medios diferentes a tarjeta de crédito son extranjeros**.

In [40]:
customer_pay_by_other = payments_rsv[payments_rsv['Tipo'] != 'Tarjeta De Crédito']
customer_pay_by_other_agency = \
customer_pay_by_other.groupby(['agency', 'customer_country'])\
                                    [['Cantidad', 'los', 'bWindow', 'adr', 'occupancy']].\
                                    agg({'Cantidad':'sum', 'los':'mean', 'bWindow':'mean',\
                                         'adr':'mean', 'occupancy':'mean'})\
                                        .sort_values(by='Cantidad', ascending=False)
customer_pay_by_other_agency['%'] = (customer_pay_by_other_agency['Cantidad']\
                                           / customer_pay_by_other_agency['Cantidad'].sum())*100
customer_pay_by_other_agency = customer_pay_by_other_agency.\
                                     reindex(columns=['Cantidad', '%', 'los', 'bWindow', 'adr', 'occupancy']) 
customer_pay_by_other_agency.style.format({"Cantidad":"${:20,.0f}", "adr":"${:20,.0f}",\
                        "%":"{:20,.2f}%", "los":"{:20,.1f}", "bWindow":"{:20,.1f}", "occupancy":"{:20,.1f}"})\
                                                .bar(subset=["Cantidad",], color='lightgreen')

#### **Optimización de la estrategia**
Ahora sabemos que el 20% de las reservas son pagadas por medio de tarjeta de crédito y que la mayoría de estas proviene de un huésped extranjero, por lo que para optimizar la estrategia haremos lo siguiente:

1. Aplicar una política de cobro por anticipado por tarjeta de crédito solo para los huéspedes extranjeros;
1. Hacer esta implementación en la tarifa más económica y medir los resultados.  De ser positivos extender su uso a todas las tarifas;
1. Aplicar la nueva política en la tarifa más económica para todas las nacionalidades en periodos de reserva futuros, tres (3) meses más adelante.  Observar los cambios, en caso que sean positivos, implementar definitivamente la nueva política en todas las tarifas para todas las nacionalidades;
1. Desarrollar un proyecto de *Machine Learning* de tipo "Aprendizaje Supervisado" con el objetivo de identificar las reservas que van a ser canceladas desde el mismo momento de su creación.  Que abran el espacio para crear iniciativas que eviten la cancelación o habiliten la reposición antes de que ocurra.